In [1]:
import requests
import csv
import json
import re
import ast

In [2]:
url = 'https://nookplaza.net/main.dart.js?v=3'
r = requests.get(url)
mainJs = r.text

In [3]:
# 'C.Iz=new H.b(13,{},C.a,[P.a,P.a])'
m = re.findall('new H\.b\(13,(.*),C\.a,\[P\.a,P\.a\]\)', mainJs)
print(len(m))

m = [item.encode("utf-8").decode('unicode-escape') for item in m]
m = [ast.literal_eval(item) for item in m if not 'null' in item]
print(len(m))

nameMap = {}
for item in m:
    nameMap[item["en-us"].lower()] = item

5715
5713


In [4]:
nameMap['street organ']

{'en-us': 'street organ',
 'en-gb': 'street organ',
 'de-de': 'Drehorgel',
 'es-es': 'organillo en carro',
 'fr-fr': 'orgue de barbarie',
 'fr-ca': 'orgue de barbarie',
 'it-it': 'organo a rullo',
 'nl-nl': 'draaiorgel',
 'zh-cn': '手摇风琴',
 'zh-tw': '手搖風琴',
 'ja-jp': 'てまわしオルガン',
 'ko-kr': '배럴 오르간',
 'ru-ru': 'шарманка'}

In [5]:
url = 'https://api.nookplaza.net/items?category=Furniture'
r = requests.get(url)
data = r.json()['results']

In [14]:
idMap = {}
output = [['id', '名稱(英文)', '名稱(中文)', '買價', '賣價', '分類', '標籤', '主題', '可互動', '大小', '取得方式', 'DIY', '款式改造', '樣式改造', '圖片', '款式', '樣式']]
for item in data:
    if item["category"] == "Art":
        continue
    name = nameMap[item['name'].lower()]
    nameMap[item['name'].lower()]['id'] = item['content']['internalID']
    idMap[item['content']['internalID']] = item
    
    # 名稱 買價 賣價
    row = [int(item['content']['internalID']), name['en-us'], name['zh-tw'], item['content']['buy'], item['content']['sell']]

    # 分類 標籤 主題
    # 'category' 'tag' 'themes'
    if "None" in item['content']['themes']:
        item['content']['themes'].remove("None")
    row += [item['content']['category'], item['content']['tag'], '、'.join(item['content']['themes'])]
    
    # 互動 大小
    row += [item['content']['interact'], '{}x{}'.format(item['content']['size']['rows'], item['content']['size']['cols'])]

    # 取得方式 DIY
    if not 'dIY' in item['content']:
        print(item)
    row += ['、'.join(item['content']['obtainedFrom']), item['content']['dIY']]
        
    # 改造-款式 樣式
    # bodyCustomize bodyColor bodyTitle; patternCustomize patternColor patternTitle
    row += [item['content']['bodyCustomize'], item['content']['patternCustomize']]

    # 圖片檔名
    row += [item['content']['filename']]

    bodys = set()
    pattrens = set()
    for variation in item['variations']:
        content = variation['content']
        filename = content['filename']
        body, pattren = filename.split('_')[-2:]
        bodys.add(body)
        pattrens.add(pattren)

        # FtrKatakatatoy_Remake_0_0
        pass
    
    bodys = list(bodys)
    pattrens = list(pattrens)
    row += ['body:{},pattrens:{}'.format(','.join(bodys), ','.join(pattrens))]
       
#     print(row)
    output.append(row)
output[1:] = sorted(output[1:], key=lambda k: k[0])

In [15]:
with open('items.csv', 'wt', encoding='utf-8', newline='') as fout:
    csvout = csv.writer(fout)
    csvout.writerows(output)